# Projet Notations de films 

Introduction ? 

### importation des librairies et fichiers

In [1]:
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
!pip install lxml
!pip install aiohttp
!pip install nest_asyncio

import asyncio
import aiohttp
import pandas as pd
from bs4 import BeautifulSoup
import re
import nest_asyncio


usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: kernel kernelspec migrate run troubleshoot

Jupyter command `jupyter-nbextension` not found.


In [2]:
# Importation des librairies
import csv

# import seaborn as sns #pas besoin si utilise fonctions
import matplotlib.pyplot as plt

# Regression
import statsmodels.api as sm
from statsmodels.formula.api import rlm 

In [3]:
import os

# Importation des fichiers python (contenant des fonctions) crées dans le cadre des 3 parties du projet :

for f in os.listdir('A_Base_de_données'):
    if f.endswith('.py') and f != 'web_scrapping.py': # A gérer !!!!!
        module_name = os.path.splitext(f)[0]
        exec(f"from A_Base_de_données.{module_name} import *")


for f in os.listdir('B_Analyse_descriptive'):
    if f.endswith('.py'):
        module_name = os.path.splitext(f)[0]
        exec(f"from B_Analyse_descriptive.{module_name} import *")


for f in os.listdir('C_Modele'):
    if f.endswith('.py'):
        module_name = os.path.splitext(f)[0]
        exec(f"from C_Modele.{module_name} import *")


## I - Création de la base de données

### Webscrapping AlloCiné

In [ ]:
# Récupération d'une liste de films qui possèdent tous une note moyenne de spectateur et une note moyenn de presse entre l'année .... et l'année .....
base_films = get_base_films(2010, 2023)


# Récupération pour chaque film de différentes caractériques
nest_asyncio.apply()
base_films_enrichie, presse = get_carac_film(base_films)


In [ ]:
# Exportation en CSV => mettre en commentaire lors du rendu final afin que cela soit toujours le même CSV qui soit utilisé par la suite
from pathlib import Path 
filepath = Path('/home/onyxia/work/Projet_Cin-/A_Base_de_données/base_allociné_2010_2023.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
base_films_enrichie.to_csv(filepath)

In [4]:
# Lecture des bases CSV : 

# Base de films : contenant diverses caractéristiques sur les films
base_films = pd.read_csv('A_Base_de_données/base_films_2010_2023.csv')


# Base presse : contenant la note de chaque presse pour chaque film
presse = pd.read_csv('A_Base_de_données/presse_2010_2023.csv')


### Traitement et création de variables

Base presse

In [6]:
# Traduction des notes en numérique (Très bien => 4, etc)
presse_notes = equivalence_notes(presse)
presse_notes.head()

/home/onyxia/work/Projet_Cin-/A_Base_de_données/mise_en_forme.py:94: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  notes = dataframe.replace(equivalences)


,identifiant,L'Express,Télérama,Brazil,Les Fiches du Cinéma,Les Inrockuptibles,Le Figaroscope,Cahiers du Cinéma,Le Point,Bande à part,...,Libération,Slate,Voici,Studio Magazine,Studio Ciné Live,Dernières Nouvelles d'Alsace,Le Soir,L'Ecran Fantastique,Culturopoing.com,aVoir-aLire.com
0,173528,3,1,NaN,NaN,1,1.0,1,3,NaN,...,1,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN
1,146632,2,2,3.0,NaN,1,5.0,1,4,NaN,...,1,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN
2,133588,3,3,NaN,NaN,3,4.0,NaN,3,NaN,...,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,132039,3,4,3.0,NaN,4,3.0,2,5,NaN,...,3,NaN,NaN,NaN,5,NaN,NaN,2.0,NaN,NaN
4,123534,5,3,4.0,NaN,NaN,4.0,NaN,3,NaN,...,NaN,NaN,NaN,NaN,4,NaN,NaN,4.0,NaN,NaN


base films

In [21]:
# Ajout du genre des réalisateurs
table_films = get_genre_individuel(base_films, 'auteur')

# Aperçu du nb de réalisateurs m vs f vs autres
table_films['genre_ind'].value_counts()

genre_ind
m      3946
f       901
m,f     211
f,m      50
Name: count, dtype: int64

In [22]:
# Conversion des notes en décimale
table_films['spectateur'] = table_films['spectateur'].apply(mise_en_forme_decimale)
table_films['presse'] = table_films['presse'].apply(mise_en_forme_decimale)

# Création variable : différence de notation entre spectateur et presse
table_films['diff_notation'] = table_films['spectateur']-table_films['presse']

In [23]:
# Création de la variable année
table_films = get_annee(table_films, 'date')

In [24]:
# Conversion de la durée en minutes
table_films['durée'] = table_films['durée'].astype('string')
table_films['duree_min'] = table_films['durée'].apply(duree_en_minutes)

In [25]:
table_films.head()

,genre_ind,titre,identifiant,date,durée,auteur,spectateur,presse,genre1,genre2,...,budget,langues,format_production,couleur,format_audio,format_projection,num_visa,diff_notation,annee,duree_min
0,m,Le Grand jeu,233140,3 janvier 2018,2h 20min,Aaron Sorkin,3.9,3.5,Biopic,Drame,...,-,Anglais,-,Couleur,-,-,147906,0.4,2018,140.0
1,m,Les Sept de Chicago,130085,16 octobre 2020,2h 09min,Aaron Sorkin,4.2,4.4,Drame,Historique,...,-,Anglais,-,Couleur,-,-,-,-0.2,2020,129.0
2,m,USS Greyhound - La bataille de l'Atlantique,250108,10 juillet 2020,1h 31min,Aaron Schneider,3.6,2.7,Drame,Historique,...,-,Anglais,-,Couleur,-,-,-,0.9,2020,91.0
3,m,Being the Ricardos,289496,21 décembre 2021,2h 05min,Aaron Sorkin,3.3,3.6,Biopic,Drame,...,-,Anglais,-,Couleur,-,-,-,-0.3,2021,125.0
4,m,Le Secret de la cité perdue,287082,20 avril 2022,1h 52min,Aaron Nee,2.4,2.7,Action,Aventure,...,-,Anglais,-,Couleur,-,-,156428,-0.3,2022,112.0


## II -  Analyse descriptive

### 1) Analyses descriptives générales sur les films de toutes nationalités

### Base CNC : Analyse du type de devis, obtenus par les réalisateurs vs les réalisatrices

Une base du CNC nous donne ce type d'information : les sociétés productrices et distributrices des films, chaînes de télévision partenaires en préachat. Je veux étudier les différences de montant des devis selon le genre du réalisateur.


In [6]:
#Ici, j'importe les données du CNC en utilisant un API.
import requests
import pandas as pd
api_root = "https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/cnc-production-cinematographique-liste-des-films-agrees/exports/json?limit=-1"
response = requests.get(api_root)
films_cnc_brut = response.json() #je mets tout dans un dico json
print(response.content[:1000])

films_cnc = pd.json_normalize(films_cnc_brut) 
films_cnc.head(10)

b'[{"visa": "117042", "titre": "BYE BYE BLONDIE", "realisateur": "DESPENTES Virginie", "producteurs": "MASTER MOVIES / RED STAR CINEMA / WILD BUNCH", "devis": 4103906.0, "genre": "Fiction", "eof": null, "rang": "2", "asr": "avant", "payante": null, "clair": null, "nationalite": ["France", "Belgique", "Suisse"], "date": "2010", "decomposition": "France-65 / Belgique-21 / Suisse-14", "aides": ["Sofica"]},{"visa": "127028", "titre": "DE L\'HUILE SUR LE FEU", "realisateur": "BENAMOU Nicolas", "producteurs": "AXEL FILMS / RECIFILMS / SARL PASTRAMI CORP / TF1 DROITS AUDIOVISUELS / UGC IMAGES", "devis": 3320336.0, "genre": "Fiction", "eof": null, "rang": "1", "asr": null, "payante": null, "clair": null, "nationalite": ["France"], "date": "2010", "decomposition": "France-100", "aides": ["Sofica", "Cr\\u00e9dit D\'impot"]},{"visa": "124421", "titre": "DES JEUNES GENS MODERNES", "realisateur": "DE MISSOLZ J\\u00e9r\\u00f4me", "producteurs": "ARTE FRANCE CINEMA / LOVE STREAMS AGNES B. PRODUCTIONS

,visa,titre,realisateur,producteurs,devis,genre,eof,rang,asr,payante,clair,nationalite,date,decomposition,aides
0,117042,BYE BYE BLONDIE,DESPENTES Virginie,MASTER MOVIES / RED STAR CINEMA / WILD BUNCH,4103906.0,Fiction,None,2,avant,None,None,"[France, Belgique, Suisse]",2010,France-65 / Belgique-21 / Suisse-14,[Sofica]
1,127028,DE L'HUILE SUR LE FEU,BENAMOU Nicolas,AXEL FILMS / RECIFILMS / SARL PASTRAMI CORP / ...,3320336.0,Fiction,None,1,None,None,None,[France],2010,France-100,"[Sofica, Crédit D'impot]"
2,124421,DES JEUNES GENS MODERNES,DE MISSOLZ Jérôme,ARTE FRANCE CINEMA / LOVE STREAMS AGNES B. PRO...,815248.0,Documentaire,None,2,None,None,None,[France],2010,France-100,[Crédit D'impot]
3,123383,EXERCICE DE L'ETAT (L'),SCHOELLER Pierre,ARCHIPEL 35 / FRANCE 3 CINEMA,6548121.0,Fiction,None,3,avant,None,None,"[France, Belgique]",2010,France-80 / Belgique-20,"[Sofica, Crédit D'impot]"
4,124350,MA PART DU GATEAU,KLAPISCH Cédric,CE QUI ME MEUT - MOTION PICTURES / FRANCE 2 CI...,7885486.0,Fiction,None,3,None,None,None,[France],2010,France-100,"[Sofica, Crédit D'impot, Aide Régionale]"
5,122385,MARQUIS (LE),FARRUGIA Dominique,FARRUDG ENTERTAINMENT WORLDWIDE / PATHE PRODUC...,8880926.0,Fiction,None,3,None,None,None,[France],2010,France-100,[Crédit D'impot]
6,126154,PROIE (LA),VALETTE Eric,BRIO FILMS / STUDIOCANAL / TF1 FILMS PRODUCTION,9706711.0,Fiction,None,3,None,None,None,[France],2010,France-100,"[Sofica, Crédit D'impot, Aide Régionale]"
7,125930,UNE EDUCATION NORVEGIENNE,LIEN Jens,LES FILMS D'ANTOINE,3214636.0,Fiction,None,3,None,None,None,"[Norvege, Suede, France, Danemark]",2010,Norvege-66 / Suede-13 / France-10 / Danemark-10,None
8,115135,VOIE RAPIDE,RODRIGUEZ Christophe,SESAME FILMS,849880.0,Fiction,None,1,None,None,None,[France],2010,France-100,"[Sofica, Aide Régionale]"
9,122784,VOYEZ COMME ILS DANSENT,MILLER Claude,ALICELEO - AUTEURS LIVRES SCENARIO POUR L'ECRI...,10492854.0,Fiction,None,3,None,None,None,"[France, Canada]",2010,France-85 / Canada-15,[Sofica]


In [7]:
#Création d'une typologie des devis (je reprends les critères de l'article)
films_cnc["type_de_devis"] = pd.cut(films_cnc.devis, [0,2000000,4000000,7000000, 16000000], right=False)
films_cnc["type_de_devis"]= films_cnc["type_de_devis"].cat.rename_categories(["catégorie 1", "catégorie 2", "catégorie 3", "catégorie 4"])
films_cnc.head()


,visa,titre,realisateur,producteurs,devis,genre,eof,rang,asr,payante,clair,nationalite,date,decomposition,aides,type_de_devis
0,117042,BYE BYE BLONDIE,DESPENTES Virginie,MASTER MOVIES / RED STAR CINEMA / WILD BUNCH,4103906.0,Fiction,None,2,avant,None,None,"[France, Belgique, Suisse]",2010,France-65 / Belgique-21 / Suisse-14,[Sofica],catégorie 3
1,127028,DE L'HUILE SUR LE FEU,BENAMOU Nicolas,AXEL FILMS / RECIFILMS / SARL PASTRAMI CORP / ...,3320336.0,Fiction,None,1,None,None,None,[France],2010,France-100,"[Sofica, Crédit D'impot]",catégorie 2
2,124421,DES JEUNES GENS MODERNES,DE MISSOLZ Jérôme,ARTE FRANCE CINEMA / LOVE STREAMS AGNES B. PRO...,815248.0,Documentaire,None,2,None,None,None,[France],2010,France-100,[Crédit D'impot],catégorie 1
3,123383,EXERCICE DE L'ETAT (L'),SCHOELLER Pierre,ARCHIPEL 35 / FRANCE 3 CINEMA,6548121.0,Fiction,None,3,avant,None,None,"[France, Belgique]",2010,France-80 / Belgique-20,"[Sofica, Crédit D'impot]",catégorie 3
4,124350,MA PART DU GATEAU,KLAPISCH Cédric,CE QUI ME MEUT - MOTION PICTURES / FRANCE 2 CI...,7885486.0,Fiction,None,3,None,None,None,[France],2010,France-100,"[Sofica, Crédit D'impot, Aide Régionale]",catégorie 4


à noter : dans l'article, ils créent une cinquième catégorie rassemblant "les sociétés de production et les sociétés de distribution qui n’ont produit ou distribué qu’un ou deux longs-métrages d’initiative française sur la période de trois ans étudiée. Cette catégorie 9 regroupe donc des producteurs et des distributeurs qui ont eu une très faible activité sur la période (plusieurs ont eu une courte durée de vie) ou encore des distributeurs liés à des majors américaines pour lesquels la distribution de FIF est une activité marginale. 272 films ont été produits et 70 distribués par de telles sociétés."

Par ailleurs, on peut également faire une typologie en fonction du type de distributeur (voir l'article également).


J'essaie ici de préciser le genre du réalisateur. Dans le cas où le réalisateur est un homme, j'attribue à ma variable genre "m", dans le cas où c'est une femme, j'attribue à ma variable genre "f". Si c'est les deux, "plusieurs réalisateurs".

Je rencontre plusieurs difficultés : déjà j'ai plusieurs réalisateurs par films parfois. J'ai donc essayer de créer une fonction qui me crée une variable par réalisateur (j'ai un petit problème ici). Ensuite l'idée est de créer la variable genre décrite ci-dessus, puis de passer aux analyses.

In [8]:
films_cnc = films_cnc.set_index('visa')
films_cnc

,titre,realisateur,producteurs,devis,genre,eof,rang,asr,payante,clair,nationalite,date,decomposition,aides,type_de_devis
visa,,,,,,,,,,,,,,,
117042,BYE BYE BLONDIE,DESPENTES Virginie,MASTER MOVIES / RED STAR CINEMA / WILD BUNCH,4103906.0,Fiction,None,2,avant,None,None,"[France, Belgique, Suisse]",2010,France-65 / Belgique-21 / Suisse-14,[Sofica],catégorie 3
127028,DE L'HUILE SUR LE FEU,BENAMOU Nicolas,AXEL FILMS / RECIFILMS / SARL PASTRAMI CORP / ...,3320336.0,Fiction,None,1,None,None,None,[France],2010,France-100,"[Sofica, Crédit D'impot]",catégorie 2
124421,DES JEUNES GENS MODERNES,DE MISSOLZ Jérôme,ARTE FRANCE CINEMA / LOVE STREAMS AGNES B. PRO...,815248.0,Documentaire,None,2,None,None,None,[France],2010,France-100,[Crédit D'impot],catégorie 1
123383,EXERCICE DE L'ETAT (L'),SCHOELLER Pierre,ARCHIPEL 35 / FRANCE 3 CINEMA,6548121.0,Fiction,None,3,avant,None,None,"[France, Belgique]",2010,France-80 / Belgique-20,"[Sofica, Crédit D'impot]",catégorie 3
124350,MA PART DU GATEAU,KLAPISCH Cédric,CE QUI ME MEUT - MOTION PICTURES / FRANCE 2 CI...,7885486.0,Fiction,None,3,None,None,None,[France],2010,France-100,"[Sofica, Crédit D'impot, Aide Régionale]",catégorie 4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150436,Garder ton nom,DUQUESNE Vincent,ELÉMIAH,1834218.0,Fiction,OUI,1,None,[C+],None,[France],2019,France-100,[Crédit D'impot],catégorie 1
149831,J'irai ou tu iras,NAKACHE Géraldine,NORD-OUEST FILMS / MARS CINÉMA,5465545.0,Fiction,OUI,3,None,"[C+, Ciné+]",F2,[France],2019,France-100,"[Sofica, Crédit D'impot]",catégorie 3
148802,Maria,TY Theodore,SIMPATICO (MAIN JOURNEY),27614575.0,Animation,OUI,1,None,[C+],M6 W9,"[Canada, France]",2019,Canada-76 / France-24,None,NaN


In [ ]:
#à affiner : créer une colonne réalisateur 2
test = films_cnc["realisateur"].str.split(pat= '/', expand=True) 
test #j'obtiens le nombre de films réalisés par plusieurs réalisateurs

0     2818
1      306
2       21
3        7
4        6
5        5
6        5
7        3
8        1
9        1
10       1
11       1
12       1
dtype: int64


In [10]:
for i in films_cnc["realisateur"]:
   realisateur_split = films_cnc["realisateur"].str.split(pat='/', expand=True, regex=True)
   
   if j in range(1,12):
     realisateur_split["j"]= None


NameError: name 'j' is not defined

In [9]:
realisateur_split = films_cnc["realisateur"].str.split(pat='/', expand=True, regex=True)
realisateur_split

,0,1,2,3,4,5,6,7,8,9,10,11,12
visa,,,,,,,,,,,,,
117042,DESPENTES Virginie,None,None,None,None,None,None,None,None,None,None,None,None
127028,BENAMOU Nicolas,None,None,None,None,None,None,None,None,None,None,None,None
124421,DE MISSOLZ Jérôme,None,None,None,None,None,None,None,None,None,None,None,None
123383,SCHOELLER Pierre,None,None,None,None,None,None,None,None,None,None,None,None
124350,KLAPISCH Cédric,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150436,DUQUESNE Vincent,None,None,None,None,None,None,None,None,None,None,None,None
149831,NAKACHE Géraldine,None,None,None,None,None,None,None,None,None,None,None,None
148802,TY Theodore,None,None,None,None,None,None,None,None,None,None,None,None


In [ ]:
for i in films_cnc["realisateur"]:
  realisateur_split = films_cnc["realisateur"].str.split(pat='/', expand=True, regex=True)  

In [48]:

def add_realisateur_columns(df):
    """
    Cette fonction crée une variable par réalisateur (quand ces derniers sont nombreux)
    """
    # Création d'un dataframe contenant un realisateur par colonne
    realisateur_split = df["realisateur"].str.split(pat='/', expand=True, regex=True)

    # Renommer les colonnes pour correspondre à "réalisateur_1", "réalisateur_2", etc.
    realisateur_split.columns = [f"réalisateur_{i+1}" for i in range(realisateur_split.shape[1])]

    # Remplacer les valeurs manquantes par 0
    realisateur_split = realisateur_split.fillna("0")

    # Ajouter les nouvelles colonnes au DataFrame original
    df = pd.concat([df, realisateur_split], axis=1)

    return df


In [49]:
films_cnc = add_realisateur_columns(films_cnc)
films_cnc

,visa,titre,realisateur,producteurs,devis,genre,eof,rang,asr,payante,...,réalisateur_4,réalisateur_5,réalisateur_6,réalisateur_7,réalisateur_8,réalisateur_9,réalisateur_10,réalisateur_11,réalisateur_12,réalisateur_13
0,117042,BYE BYE BLONDIE,DESPENTES Virginie,MASTER MOVIES / RED STAR CINEMA / WILD BUNCH,4103906.0,Fiction,None,2,avant,None,...,0,0,0,0,0,0,0,0,0,0
1,127028,DE L'HUILE SUR LE FEU,BENAMOU Nicolas,AXEL FILMS / RECIFILMS / SARL PASTRAMI CORP / ...,3320336.0,Fiction,None,1,None,None,...,0,0,0,0,0,0,0,0,0,0
2,124421,DES JEUNES GENS MODERNES,DE MISSOLZ Jérôme,ARTE FRANCE CINEMA / LOVE STREAMS AGNES B. PRO...,815248.0,Documentaire,None,2,None,None,...,0,0,0,0,0,0,0,0,0,0
3,123383,EXERCICE DE L'ETAT (L'),SCHOELLER Pierre,ARCHIPEL 35 / FRANCE 3 CINEMA,6548121.0,Fiction,None,3,avant,None,...,0,0,0,0,0,0,0,0,0,0
4,124350,MA PART DU GATEAU,KLAPISCH Cédric,CE QUI ME MEUT - MOTION PICTURES / FRANCE 2 CI...,7885486.0,Fiction,None,3,None,None,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4615,150436,Garder ton nom,DUQUESNE Vincent,ELÉMIAH,1834218.0,Fiction,OUI,1,None,[C+],...,0,0,0,0,0,0,0,0,0,0
4616,149831,J'irai ou tu iras,NAKACHE Géraldine,NORD-OUEST FILMS / MARS CINÉMA,5465545.0,Fiction,OUI,3,None,"[C+, Ciné+]",...,0,0,0,0,0,0,0,0,0,0
4617,148802,Maria,TY Theodore,SIMPATICO (MAIN JOURNEY),27614575.0,Animation,OUI,1,None,[C+],...,0,0,0,0,0,0,0,0,0,0
4618,151720,Small body,SAMANI Laura,TOMSA FILMS,2062780.0,Fiction,NON,1,None,None,...,0,0,0,0,0,0,0,0,0,0


In [52]:
films_cnc.columns

Index(['visa', 'titre', 'realisateur', 'producteurs', 'devis', 'genre', 'eof',
       'rang', 'asr', 'payante', 'clair', 'nationalite', 'date',
       'decomposition', 'aides', 'type_de_devis', 'réalisateur_1',
       'réalisateur_2', 'réalisateur_3', 'réalisateur_4', 'réalisateur_5',
       'réalisateur_6', 'réalisateur_7', 'réalisateur_8', 'réalisateur_9',
       'réalisateur_10', 'réalisateur_11', 'réalisateur_12', 'réalisateur_13',
       'réalisateur_1', 'réalisateur_2', 'réalisateur_3', 'réalisateur_4',
       'réalisateur_5', 'réalisateur_6', 'réalisateur_7', 'réalisateur_8',
       'réalisateur_9', 'réalisateur_10', 'réalisateur_11', 'réalisateur_12',
       'réalisateur_13', 'réalisateur_1', 'réalisateur_2', 'réalisateur_3',
       'réalisateur_4', 'réalisateur_5', 'réalisateur_6', 'réalisateur_7',
       'réalisateur_8', 'réalisateur_9', 'réalisateur_10', 'réalisateur_11',
       'réalisateur_12', 'réalisateur_13', 'réalisateur_1', 'réalisateur_2',
       'réalisateur_3', 'ré

In [ ]:
films_cnc["réalisateur_13"] = films_cnc["réalisateur_13"].astype(str) #je convertis tout en chaîne de caractère

In [51]:
films_cnc["réalisateur_13"].nunique()

réalisateur_13    2
réalisateur_13    2
réalisateur_13    2
réalisateur_13    2
dtype: int64

In [50]:
films_cnc.loc[films_cnc["réalisateur_13"] != "0"]


ValueError: Cannot index with multidimensional key

Je vais éliminer de l'analyse les films réalisés par plus de 3 réalisateurs. Au total, j'en ai ...

In [ ]:
#J'intègre le genre dans ma base de données
def get_genre_individuel(dataframe, colonne, position):
    """  
    """
    base_prenom_genre = base_prenom()
    base_prenom_genre = base_prenom_genre.drop(columns=['04_fréquence'])

    # transformation du prénom en minuscule
    dataframe['prenom'] = dataframe[colonne].str.split().str[position].str.lower()

    base_avec_genre = pd.merge(base_prenom_genre, dataframe, on='prenom', how='inner')
    base_avec_genre_nettoyée = base_avec_genre.drop(labels='prenom', axis='columns')
    return base_avec_genre_nettoyée



In [ ]:
films_cnc_avec_genre = get_genre_individuel(films_cnc, 'realisateur', -1)
films_cnc_avec_genre

In [ ]:
films_cnc_avec_genre.size

films_cnc_avec_genre['genre_ind'].value_counts()


### Fusion base CNC et base allociné

J'effectue un test pour voir si la base CNC et la base allociné peuvent bien être fusionnées. Je récupère donc ces informations : ttes les infos cnc, les moyennes des notes presse et spectateur allociné. 

In [19]:
#Préparation de la base allociné pour le merge 
base_films_2010 = get_base_films(2010, 2010)
nest_asyncio.apply()
base_films_enrichie_2010 = get_carac_film(base_films_2010)

base_films_enrichie_2010['num_visa'] = base_films_enrichie_2010['num_visa'].str.replace(' ', '')

nombre de films récupérés :  491


In [23]:
inner_merge_cnc_allociné = pd.merge(left = films_cnc , right = base_films_enrichie_2010 , how='inner', left_on="visa", right_on='num_visa')

In [24]:
inner_merge_cnc_allociné

,visa,titre_x,realisateur,producteurs,devis,genre,eof,rang,asr,payante,...,date_sortie_bluray,date_sortie_vod,type_film,budget,langues,format_production,couleur,format_audio,format_projection,num_visa
0,122207,AO LE DERNIER NEANDERTAL,MALATERRE Jacques,FRANCE 2 CINEMA / UGC IMAGES / UGC YM,13398412.0,Fiction,None,1,None,None,...,-,01/12/2016,Long métrage,13 000 000 EUR,Français,-,Couleur,-,-,122207
1,122530,COURSIER,RENOH Hervé,BLACK MASK PRODUCTIONS / EUROPACORP / M6 FILMS...,9410932.0,Fiction,None,2,None,None,...,-,03/12/2019,Long métrage,-,Français,-,Couleur,-,-,122530
2,121405,MEUTE (LA),RICHARD Franck,LA FABRIQUE 2,2866560.0,Fiction,None,1,None,None,...,-,01/01/2016,Long métrage,2 500 000 EUR,Français,-,Couleur,-,-,121405
3,123656,REVOLTES DE L'ILE DU DIABLE (LES),HOLST M.,MACT PRODUCTIONS,5296837.0,Fiction,None,3,None,None,...,24/10/2012,23/03/2012,Long métrage,9 500 000 USD,"Norvégien, Suédois",-,Couleur,-,-,123656
4,120962,TOURNEE,AMALRIC Mathieu,ARTE FRANCE CINEMA / LE PREMIER POISSON,3515910.0,Fiction,None,2,avant,None,...,29/09/2016,30/10/2010,Long métrage,4 150 000 EUR,"Anglais, Français",35mm,Couleur,Dolby Digital,1.85 : 1,120962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,124018,MAMMUTH,KERVERN Gustave / DELEPINE Benoît,ARTE FRANCE CINEMA / DD PRODUCTIONS / GMT PROD...,2499161.0,Fiction,None,3 / 3,None,None,...,-,15/08/2015,Long métrage,2 000 000 USD,Français,35mm,Couleur,Dolby Digital,1.85 : 1,124018
164,121731,TETE AILLEURS (LA),PELLE Frédéric,7EME APACHE FILMS / BIANCA FILMS,1054741.0,Fiction,None,1,None,None,...,-,-,Long métrage,-,Français,-,Couleur,-,-,121731
165,123205,TETE EN FRICHE (LA),BECKER Jean,DD PRODUCTIONS / FRANCE 3 CINEMA / ICE 3 / KJB...,8274491.0,Fiction,None,3,None,None,...,-,23/10/2010,Long métrage,-,"Flamand, Français",35mm,Couleur,"Dolby Digital, DTS",2.35 : 1 Cinemascope,123205
166,122965,UN HOMME QUI CRIE,HAROUN Mahamat-Saleh,PILI FILMS,2066392.0,Fiction,None,3,avant,None,...,-,29/01/2011,Long métrage,2 000 000 EUR,Français,-,Couleur,-,-,122965


## III - Modelisation